# Imports

In [8]:
import pandas as pd

# Exploration

In [15]:
# get data sets

# evictions df
edf = pd.read_excel('evictions_prepared.xlsx')

#requests df
rdf = pd.read_excel('requests_prepared.xlsx')

In [16]:
edf.head(10)

,case_number,judgement_date,zip_code,disposition
0,41E2407310,2024-12-19,78223,Judgment for Plaintiff (OCA)
1,21E2407657,2024-12-19,78229,Default Judgments (OCA)
2,21E2407655,2024-12-19,78229,Default Judgments (OCA)
3,21E2407654,2024-12-19,78229,Default Judgments (OCA)
4,21E2407650,2024-12-19,78251,Default Judgments (OCA)
5,21E2407649,2024-12-19,78229,Default Judgments (OCA)
6,21E2407648,2024-12-19,78229,Default Judgments (OCA)
7,21E2407647,2024-12-19,78229,Judgment for Plaintiff (OCA)
8,21E2407646,2024-12-19,78229,Judgment for Plaintiff (OCA)
9,21E2407645,2024-12-19,78228,Default Judgments (OCA)


In [17]:
rdf.head(10)

,Unnamed: 0,open_date,type,latitude,longitude,zip_code
0,0,2023-10-20,Homeless Encampment,29.444219,-98.516167,78201
1,1,2023-11-02,Homeless Encampment,29.351602,-98.534868,78224
2,2,2023-11-06,Homeless Encampment,29.535004,-98.541934,78230
3,3,2023-11-28,Homeless Encampment,29.461083,-98.569024,78228
4,4,2023-12-04,Homeless Encampment,29.528313,-98.556656,78230
5,5,2023-12-06,Homeless Encampment,29.512414,-98.525528,78213
6,6,2023-12-16,Homeless Encampment,29.507367,-98.519163,78213
7,7,2023-12-19,Homeless Encampment,29.621459,-98.464573,78259
8,8,2023-12-19,Homeless Encampment,29.343525,-98.484157,78214
9,9,2023-12-20,Homeless Encampment,29.317004,-98.503728,78221


## What is the time overlap between the two data sets?
* Evictions data seems to cover the whole of 2024
* Requests data seems to cover from late 2023 to early 2025
* an apt comparison should be able to be made using the full eviction data and reqiests made during 2024

In [20]:
print(f'The eviction dataframe contains data from') 
print(f'{edf.judgement_date.min()}')
print(f'to') 
print(f'{edf.judgement_date.max()}')
print()
print(f'The requests dataframe contains data from') 
print(f'{rdf.open_date.min()}')
print(f'to') 
print(f'{rdf.open_date.max()}')

The eviction dataframe contains data from
2024-01-02 00:00:00
to
2024-12-19 00:00:00

The requests dataframe contains data from
2023-10-20 00:00:00
to
2025-04-12 00:00:00


In [24]:
rdf = rdf[rdf['open_date'].dt.year == 2024]

rdf.shape

(2197, 6)

In [23]:
rdf.head()

,Unnamed: 0,open_date,type,latitude,longitude,zip_code
12,12,2024-01-03,Homeless Encampment,29.427353,-98.503727,78207
13,13,2024-01-03,Homeless Encampment,29.448031,-98.707380,78251
14,14,2024-01-04,Homeless Encampment,29.524882,-98.410335,78217
15,15,2024-01-04,Homeless Encampment,29.428445,-98.452225,78202
16,16,2024-01-05,Homeless Encampment,29.476837,-98.588266,78228


## Is there a correlation between the number of eviction cases and homelessness cases in a given zip code during that time?